# **Установим необходимые библиотеки**

In [1]:
!pip install ipywidgets
!pip install pulp
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 22.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Сама программа**

In [8]:
import ipywidgets as widgets
import numpy as np
import io
import uuid
import pandas as pd
import google.colab.data_table as dt
from pulp import *
from IPython.display import display
from IPython.display import HTML
from google.colab import files
from google.colab import output


accordion = widgets.Accordion()
USERDATA_DATAFRAME = None


def compute_computer_assembly(amount_of_weeks: int, demand_matrix: [], producing_prices: [],
                              storage_price: [], testing_times: [], product_volumes: [],
                              storage_volume: int, max_assemblies: int, max_test_time: int, transpose=False) -> tuple:
    """Finds the best solution """
    # Initializing Problem
    model = LpProblem("Computers_Assembly", LpMinimize)

    # Initializing solution array
    x = [[LpVariable(f"x_{i}_{j}", lowBound=0, cat="Integer") for j in range(3)] for i in range(amount_of_weeks)]


    # Initializing I matrix
    eow_left = [[LpVariable(f"I_{i}_{j}", lowBound=0, cat="Integer") for j in range(3)] for i in range(amount_of_weeks)]
    for i in range(1, amount_of_weeks):
        for j in range(3):
            eow_left[i][j] = LpVariable(f"I_{i}_{j}", lowBound=0, cat="Integer")
            if demand_matrix[i][j] == 0:
                model += eow_left[i][j] == 0
            else:
                # Otherwise, update eow_left[i][j] based on the formula you provided
                model += eow_left[i][j] == x[i][j] - demand_matrix[i][j] + eow_left[i-1][j]



    # Setting up the minimization function
    model += lpSum(
        producing_prices[i][j] * x[i][j] + storage_price[i][j] * eow_left[i][j] for i in range(amount_of_weeks) for j in
        range(3))

    # Setting up the first restriction
    for i in range(amount_of_weeks):
        for j in range(3):
            model += x[i][j] + eow_left[i - 1][j] >= demand_matrix[i][j] + eow_left[i][j]


    # Setting up the max assemblies limit
    for i in range(amount_of_weeks):
        model += lpSum(x[i][j] for j in range(3)) <= max_assemblies

    # Setting up the test time limits
    for i in range(amount_of_weeks):
        model += lpSum(testing_times[j] * x[i][j] for j in range(3)) <= max_test_time

    # Setting up the storage volume restriction
    for i in range(amount_of_weeks):
        model += lpSum(product_volumes[j] * x[i][j] for j in range(3)) <= storage_volume

    result = model.solve(PULP_CBC_CMD(msg=False))

    if result != LpStatusInfeasible:
        result_array = []
        for i in range(amount_of_weeks):
            current_line = []
            for j in range(3):
                current_line.append(int(value(x[i][j])))
            result_array.append(current_line)
        if transpose:
            result_array = np.transpose(result_array)
        return value(model.objective), result_array, "The solution was successfully found!"
    return None, None, "Unable to find the solution!"




def upload_file(change):
    """Handles the input data file upload"""
    global USERDATA_DATAFRAME
    uploaded_files = change['owner']
    name, data = list(uploaded_files.value.items())[0]
    if 'xl' in name:
        temp =  pd.ExcelFile(io.BytesIO(data['content']))
        if len(temp.sheet_names) < 3:
          output.eval_js('alert("Недостаточно страниц в Excel документе! Необходимо как минимум 3!")')
        else:
          USERDATA_DATAFRAME = temp
          sheets_concat = "Обнаружены листы: "
          for i in USERDATA_DATAFRAME.sheet_names:
            sheets_concat += i + ", "
          print(sheets_concat[:-1])
    else:
        # Unsupported file type
        output.eval_js('alert("Неподдерживаемый тип файла! Нужен файл Microsoft Office Excel!")')

def download_excel(df):
  """Converts a DataFrame to the Excel file and downloads it"""
  buffer = BytesIO()
  df.to_excel(buffer)
  buffer.seek(0)


def process_calculation(button, planned_weeks:int, max_builds:int, storage_volume: int, test_times: [], max_test_time: int, product_volumes: [], demand_sheet: str, costs_sheet: str, storage_sheet: str):
    """"Handles the calculate button click"""
    global USERDATA_DATAFRAME
    global accordion
    if planned_weeks <= 0 or max_builds <= 0 or test_times[0] <= 0 or test_times[1] <= 0 or test_times[2] <= 0 or product_volumes[0] <=0 or product_volumes[1] <= 0 or product_volumes[2] <= 0 or not len(demand_sheet) or not len(costs_sheet) or not len(storage_sheet):
      output.eval_js('alert("Нужно заполнить все поля! Все целочисленные знаечния >0, все стрковые имеют длину > 0 !")')
      return 0
    if USERDATA_DATAFRAME is None:
      output.eval_js('alert("Файл с данными не обнаружен! Необходимо загрузить файл Microsoft Office Excel!")')
      return 0

    demand_matrix = np.transpose(USERDATA_DATAFRAME.parse(demand_sheet, header=None).values.tolist())
    costs_matrix = np.transpose(USERDATA_DATAFRAME.parse(costs_sheet, header=None).values.tolist())
    storage_matrix = np.transpose(USERDATA_DATAFRAME.parse(storage_sheet, header=None).values.tolist())
    
    calculated_cost, table, message  = compute_computer_assembly(planned_weeks, demand_matrix, costs_matrix, storage_matrix, test_times, product_volumes, storage_volume, max_builds, max_test_time, True )
    if table is None:
      output.eval_js('alert("Оптимальное решение для данной конфигурации не было найдено! Ah crap, Im sorry!")')
    else:
      weeks = []
      rownames = ["ПК", "Ноутбуки", "Мейнфреймы"]
      for i in range(1, planned_weeks+1):
        weeks.append(f"Неделя №{i}")
      df = pd.DataFrame(table, columns=weeks, index=rownames)
      out = widgets.Output()
      with out:
          display(df.style.set_table_styles([{'selector': 'th', 'props': [('background', '#607c8e'), ('color', 'white')]}]))
      title = widgets.HTML(f'<h1 style="font-weight:bold; font-size:22px">Результаты Расчетов</h1>')    
      text = widgets.HTML(f'<h1 style="font-weight:bold; font-size:18px">Затраты: {calculated_cost}</h1>')
      form = widgets.VBox([title, text, out])
      accordion.children += (form, )
      df = pd.DataFrame(table, columns=weeks, index=rownames)
      path = f'computertask_resultdata_{str(uuid.uuid4())}.xlsx'
      writer = pd.ExcelWriter(path)
      df.to_excel(writer, index=False)
      writer.save()
      files.download(path)
  




def init_gui():
  # Main Parameters
  logo = widgets.HTML(f'<h1 style="font-weight:bold; font-size:18px">Задача управления запасами. Ввод данных.</h1>')
  planned_weeks = widgets.IntText(description='Планируемое количество недель:', style={'description_width': 'initial'})
  max_builds = widgets.IntText(description='Максимальное количество сборок:', style={'description_width': 'initial'})
  storage_volume = widgets.IntText(description='Объем склада:', style={'description_width': 'initial'})
  max_test_time = widgets.IntText(description='Максимальное время тестирований:', style={'description_width': 'initial'})

  # Test times
  test_time_I = widgets.IntText(description='Время тестирования продукции I:', style={'description_width': 'initial'})
  test_time_II = widgets.IntText(description='Время тестирования продукции II:', style={'description_width': 'initial'})
  test_time_III = widgets.IntText(description='Время тестирования продукции III:', style={'description_width': 'initial'})

  # Products Volume
  product_volume_I = widgets.IntText(description='Объем продукции I:', style={'description_width': 'initial'})
  product_volume_II = widgets.IntText(description='Объем продукции II:', style={'description_width': 'initial'})
  product_volume_III = widgets.IntText(description='Объем продукции III:', style={'description_width': 'initial'})

  # Information from the file
  load_button = widgets.FileUpload(description='Загрузить данные из Excel', style={'description_width': 'initial'})
  load_button.observe(upload_file,  names='_counter')
  demand_sheet = widgets.Text(description='Название листа с данными о потребности:', layout=widgets.Layout(width='700px', height='20px', margin='30px 0 0 0'), style={'description_width': 'initial'})
  production_cost_sheet = widgets.Text(description='Название листа с данными о стоимости производства:', layout=widgets.Layout(width='700px', height='20px', margin='30px 0 0 0'), style={'description_width': 'initial'})
  storage_cost_sheet = widgets.Text(description='Название листа с данными о стоимости хранения:', layout=widgets.Layout(width='700px', height='20px', margin='30px 0 0 0'), style={'description_width': 'initial'})

  # Calculate button
  calculate_button = widgets.Button(description='Рассчитать', button_style='primary')

  # Group parameters by sections
  general_params = widgets.VBox([planned_weeks, max_builds, storage_volume, max_test_time], layout=widgets.Layout(padding='10px'))
  test_time_params = widgets.VBox([test_time_I, test_time_II, test_time_III], layout=widgets.Layout(padding='10px'))
  product_volume_params = widgets.VBox([product_volume_I, product_volume_II, product_volume_III], layout=widgets.Layout(padding='10px'))
  file_info_params = widgets.VBox([load_button, demand_sheet, production_cost_sheet, storage_cost_sheet], layout=widgets.Layout(padding='10px'))
  button_box = widgets.HBox([calculate_button], layout=widgets.Layout(justify_content='center', padding='10px'))

  # Join parameters into the one formъ
  form = widgets.VBox([logo, general_params, test_time_params, product_volume_params, file_info_params, button_box], layout=widgets.Layout(padding='10px'))
  accordion.children += (form,)
  # Display the form
  display(accordion)

  calculate_button.on_click(lambda b: process_calculation(b, planned_weeks.value, max_builds.value, storage_volume.value, [test_time_I.value, test_time_II.value, test_time_III.value], max_test_time.value, [product_volume_I.value, product_volume_II.value, product_volume_III.value], demand_sheet.value, production_cost_sheet.value, storage_cost_sheet.value))



def main():
  init_gui()



if __name__ == '__main__':
  main()

Accordion(children=(VBox(children=(HTML(value='<h1 style="font-weight:bold; font-size:18px">Задача управления …

Обнаружены листы: Спрос, ЦеныПроизводства, Хранение,


<ipython-input-8-c2418335c40d>:143: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>